In [15]:
class popularity():
    def __init__(self):
        self.total_data = None
        self.user_id = None
        self.song_id = None
        self.popu_rec = None

    def create(self, total_data, user_id, song_id):
        self.total_data = total_data
        self.user_id = user_id
        self.song_id = song_id
        total_data_grouped = total_data.groupby([self.song_id]).agg({self.user_id: 'count'}).reset_index()
        total_data_grouped.rename(columns={'user_id': 'score'}, inplace=True)

        total_data_sort = total_data_grouped.sort_values(['score', self.song_id], ascending=[0, 1])

        total_data_sort['Rank'] = total_data_sort['score'].rank(ascending=0, method='first')

        self.popu_rec = total_data_sort.head(10)

    def recommend(self, user_id):
        user_rec = self.popu_rec

        user_rec['user_id'] = user_id

        cols = user_rec.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_rec = user_rec[cols]

        return user_rec


In [16]:
class similarity():
    def __init__(self):
        self.total_data = None
        self.user_id = None
        self.song_id = None
        self.correl_matrix = None
        self.songs_dic = None
        self.rev_songs_dic = None
        self.song_similarity = None

    def get_user_items(self, u):
        u_data = self.total_data[self.total_data[self.user_id] == u]
        u_items = list(u_data[self.song_id].unique())

        return u_items

    def get_song_users(self, i):
        i_data = self.total_data[self.total_data[self.song_id] == i]
        i_users = set(i_data[self.user_id].unique())

        return i_users

    def get_all_items_total_data(self):
        all_items = list(self.total_data[self.song_id].unique())

        return all_items

    def construct_correl_matrix(self, u_songs, a_songs):

        u_songs_users = []
        for i in range(0, len(u_songs)):
            u_songs_users.append(self.get_song_users(u_songs[i]))

        correl_matrix = np.matrix(np.zeros(shape=(len(u_songs), len(a_songs))), float)

        for i in range(0, len(a_songs)):
            songs_i_data = self.total_data[self.total_data[self.song_id] == a_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())

            for j in range(0, len(u_songs)):

                users_j = u_songs_users[j]

                users_intersection = users_i.intersection(users_j)

                if len(users_intersection) != 0:

                    users_union = users_i.union(users_j)

                    correl_matrix[j, i] = float(len(users_intersection)) / float(len(users_union))
                else:
                    correl_matrix[j, i] = 0

        return correl_matrix

    def generate_top_r(self, user, cooccurence_matrix, a_songs, u_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))

        user_similarity_scores = cooccurence_matrix.sum(axis=0) / float(cooccurence_matrix.shape[0])
        user_similarity_scores = np.array(user_similarity_scores)[0].tolist()

        s_index = sorted(((e, i) for i, e in enumerate(list(user_similarity_scores))), reverse=True)

        columns = ['user_id', 'song', 'score', 'rank']

        df1 = pandas.DataFrame(columns=columns)

        rank = 1
        for i in range(0, len(s_index)):
            if ~np.isnan(s_index[i][0]) and a_songs[s_index[i][1]] not in u_songs and rank <= 10:
                df1.loc[len(df1)] = [user, a_songs[s_index[i][1]], s_index[i][0], rank]
                rank = rank + 1

        if df1.shape[0] == 0:
            print("The current user don't have any song for similarity based recommendation model.")
            return -1
        else:
            return df1

    def create(self, total_data, user_id, song_id):
        self.total_data = total_data
        self.user_id = user_id
        self.song_id = song_id

    def recommend(self, u):

        u_songs = self.get_user_items(u)

        print("No. of songs for the user: %d" % len(u_songs))

        a_songs = self.get_all_items_total_data()

        print("No. of songs in the list: %d" % len(a_songs))

        correl_matrix = self.construct_correl_matrix(u_songs, a_songs)
        
        

        df_r = self.generate_top_r(u, correl_matrix, a_songs, u_songs)
        print('correlation matrix')
        print(correl_matrix)
        return df_r

    def similar_items(self, i_list):

        u_songs = i_list

        a_songs = self.get_all_items_total_data()

        print("no. of unique songs in the set: %d" % len(a_songs))

        correl_matrix = self.construct_correl_matrix(u_songs, a_songs)
        u = ""
        df_r = self.generate_top_r(u, correl_matrix, a_songs, u_songs)

        return df_r


In [17]:
import pandas
from sklearn.model_selection import train_test_split
import numpy as np



In [18]:

userdata = 'userdata.txt'
songsdata = 'song_data.csv'

song_df1 = pandas.read_table(userdata,header=None)
song_df1.columns = ['user_id', 'song_id', 'listen_count']

song_df2 =  pandas.read_csv(songsdata)

song_df1 = pandas.merge(song_df1, song_df2.drop_duplicates(['song_id']), on="song_id", how="left")
song_df1.head()


,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Thicker Than Water,Jack Johnson,0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999


In [19]:

print("Total no of songs:",len(song_df1))


Total no of songs: 2000000


In [20]:
song_df1 = song_df1.head(10000)

song_df1['song'] = song_df1['title'].map(str) + " - " + song_df1['artist_name']


In [21]:


song_gr = song_df1.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_gr['listen_count'].sum()
song_gr['percentage']  = song_gr['listen_count'].div(grouped_sum)*100
song_gr.sort_values(['listen_count', 'song'], ascending = [0,1])




,song,listen_count,percentage
3660,Sehr kosmisch - Harmonia,45,0.45
4678,Undo - Björk,32,0.32
5105,You're The One - Dwight Yoakam,32,0.32
1071,Dog Days Are Over (Radio Edit) - Florence + Th...,28,0.28
3655,Secrets - OneRepublic,28,0.28
...,...,...,...
5139,high fives - Four Tet,1,0.01
5140,in white rooms - Booka Shade,1,0.01
5143,paranoid android - Christopher O'Riley,1,0.01
5149,¿Lo Ves? [Piano Y Voz] - Alejandro Sanz,1,0.01


In [22]:

u = song_df1['user_id'].unique()
print("The no. of unique users:", len(u))




The no. of unique users: 365


In [23]:

train, testdata = train_test_split(song_df1, test_size = 0.20, random_state=0)
print(train.head(5))



                                       user_id             song_id  \
7389  94d5bdc37683950e90c56c9b32721edb5d347600  SOXNZOW12AB017F756   
9275  1012ecfd277b96487ed8357d02fa8326b13696a5  SOXHYVQ12AB0187949   
2995  15415fa2745b344bce958967c346f2a89f792f63  SOOSZAZ12A6D4FADF8   
5316  ffadf9297a99945c0513cd87939d91d8b602936b  SOWDJEJ12A8C1339FE   
356   5a905f000fc1ff3df7ca807d57edb608863db05d  SOAMPRJ12A8AE45F38   

      listen_count                 title  \
7389             2      Half Of My Heart   
9275             1  The Beautiful People   
2995             1     Sanctify Yourself   
5316             4     Heart Cooks Brain   
356             20                 Rorol   

                                                release      artist_name  \
7389                                     Battle Studies       John Mayer   
9275             Antichrist Superstar (Ecopac Explicit)   Marilyn Manson   
2995                             Glittering Prize 81/92     Simple Minds   
5316  Ever

In [24]:

pop_model = popularity()
pop_model.create(train, 'user_id', 'song')

user_id1 = u[5]
pop_model.recommend(user_id1)



,user_id,song,score,Rank
3194,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Sehr kosmisch - Harmonia,37,1.0
4083,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Undo - Björk,27,2.0
931,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Dog Days Are Over (Radio Edit) - Florence + Th...,24,3.0
4443,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You're The One - Dwight Yoakam,24,4.0
3034,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Revelry - Kings Of Leon,21,5.0
3189,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Secrets - OneRepublic,21,6.0
4112,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Use Somebody - Kings Of Leon,21,7.0
1207,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Fireflies - Charttraxx Karaoke,20,8.0
1577,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hey_ Soul Sister - Train,19,9.0
1626,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [25]:

user_id2 = u[8]
pop_model.recommend(user_id2)



,user_id,song,score,Rank
3194,9bb911319fbc04f01755814cb5edb21df3d1a336,Sehr kosmisch - Harmonia,37,1.0
4083,9bb911319fbc04f01755814cb5edb21df3d1a336,Undo - Björk,27,2.0
931,9bb911319fbc04f01755814cb5edb21df3d1a336,Dog Days Are Over (Radio Edit) - Florence + Th...,24,3.0
4443,9bb911319fbc04f01755814cb5edb21df3d1a336,You're The One - Dwight Yoakam,24,4.0
3034,9bb911319fbc04f01755814cb5edb21df3d1a336,Revelry - Kings Of Leon,21,5.0
3189,9bb911319fbc04f01755814cb5edb21df3d1a336,Secrets - OneRepublic,21,6.0
4112,9bb911319fbc04f01755814cb5edb21df3d1a336,Use Somebody - Kings Of Leon,21,7.0
1207,9bb911319fbc04f01755814cb5edb21df3d1a336,Fireflies - Charttraxx Karaoke,20,8.0
1577,9bb911319fbc04f01755814cb5edb21df3d1a336,Hey_ Soul Sister - Train,19,9.0
1626,9bb911319fbc04f01755814cb5edb21df3d1a336,Horn Concerto No. 4 in E flat K495: II. Romanc...,19,10.0


In [26]:


similairty_model = similarity()
similairty_model.create(train, 'user_id', 'song')


In [27]:
    
user_id1 = u[5]
user_items1 = similairty_model.get_user_items(user_id1)
print("------------------------------------------------------------------------------------")
print("Songs played by first user %s:" % user_id1)
print("------------------------------------------------------------------------------------")

for user_item in user_items1:
    print(user_item)

print("----------------------------------------------------------------------")
print("Similar songs recommended for the first user:")
print("----------------------------------------------------------------------")


similairty_model.recommend(user_id1)



------------------------------------------------------------------------------------
Songs played by first user 4bd88bfb25263a75bbdd467e74018f4ae570e5df:
------------------------------------------------------------------------------------
Just Lose It - Eminem
Without Me - Eminem
16 Candles - The Crests
Speechless - Lady GaGa
Push It - Salt-N-Pepa
Ghosts 'n' Stuff (Original Instrumental Mix) - Deadmau5
Say My Name - Destiny's Child
My Dad's Gone Crazy - Eminem / Hailie Jade
The Real Slim Shady - Eminem
Somebody To Love - Justin Bieber
Forgive Me - Leona Lewis
Missing You - John Waite
Ya Nada Queda - Kudai
----------------------------------------------------------------------
Similar songs recommended for the first user:
----------------------------------------------------------------------
No. of songs for the user: 13
No. of songs in the list: 4483
Non zero values in cooccurence_matrix :2097
correlation matrix
[[0.         0.         0.         ... 0.         0.         0.        ]
 [

,user_id,song,score,rank
0,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Superman - Eminem / Dina Rae,0.088692,1
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Mockingbird - Eminem,0.067663,2
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm Back - Eminem,0.065385,3
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,U Smile - Justin Bieber,0.064525,4
4,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Here Without You - 3 Doors Down,0.062293,5
5,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hellbound - J-Black & Masta Ace,0.055769,6
6,4bd88bfb25263a75bbdd467e74018f4ae570e5df,The Seed (2.0) - The Roots / Cody Chestnutt,0.052564,7
7,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm The One Who Understands (Edit Version) - War,0.052564,8
8,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Falling - Iration,0.052564,9
9,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Armed And Ready (2009 Digital Remaster) - The ...,0.052564,10


In [28]:

user_id2 = u[7]

user_items2 = similairty_model.get_user_items(user_id2)
print("------------------------------------------------------------------------------------")
print("Songs played by second user %s:" % user_id2)
print("------------------------------------------------------------------------------------")

for user_item in user_items2:
    print(user_item)

print("----------------------------------------------------------------------")
print("Similar songs recommended for the second user:")
print("----------------------------------------------------------------------")


similairty_model.recommend(user_id2)

------------------------------------------------------------------------------------
Songs played by second user 9d6f0ead607ac2a6c2460e4d14fb439a146b7dec:
------------------------------------------------------------------------------------
Swallowed In The Sea - Coldplay
Life In Technicolor ii - Coldplay
Life In Technicolor - Coldplay
The Scientist - Coldplay
Trouble - Coldplay
Strawberry Swing - Coldplay
Lost! - Coldplay
Clocks - Coldplay
----------------------------------------------------------------------
Similar songs recommended for the second user:
----------------------------------------------------------------------
No. of songs for the user: 8
No. of songs in the list: 4483
Non zero values in cooccurence_matrix :3429
correlation matrix
[[0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.    ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.    0.    0.   ]
 [0.125 0.    0.    ... 0.    0.    0.   ]
 [0.2   0.    0.    ... 

,user_id,song,score,rank
0,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,She Just Likes To Fight - Four Tet,0.281579,1
1,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Warning Sign - Coldplay,0.281579,2
2,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,We Never Change - Coldplay,0.281579,3
3,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Puppetmad - Puppetmastaz,0.281579,4
4,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,God Put A Smile Upon Your Face - Coldplay,0.281579,5
5,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Susie Q - Creedence Clearwater Revival,0.281579,6
6,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,The Joker - Fatboy Slim,0.281579,7
7,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,Korg Rhythm Afro - Holy Fuck,0.281579,8
8,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,This Unfolds - Four Tet,0.281579,9
9,9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,high fives - Four Tet,0.281579,10
